In [1]:
#%pip install tensorflow
import tensorflow as tf
import pandas as pd
## Load in data

tickets = pd.read_csv("master_dataset.csv")

In [8]:
import numpy as np

def bagOfWordsEncoder(vocab, input):
    ## Make an array size of the vocab
    encoderMatrix = np.zeros(len(vocab))

    ## Iterate the input strings and encode their position in the array
    for word in input:
        index = vocab[vocab['word']==word].index.values
        encoderMatrix[index] += 1
    
    if encoderMatrix.std() != 0:
        encoderMatrix = (encoderMatrix - encoderMatrix.mean()) / encoderMatrix.std()
    return encoderMatrix

## Method for getting input
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
import nltk
nltk.download("punkt")
nltk.download('wordnet')
nltk.download("stopwords")

def parse_text(text):

    ## Tokenize string into words (and punctuation)
    word_array = word_tokenize(text)
    word_array = [word.lower() for word in word_array if word.isalpha()]

    ## Filter out stop words
    stop_words = set(stopwords.words("english"))
    filtered_words = [word for word in word_array if word.casefold() not in stop_words]

    ## Turn words into lemmatized words
    lemmatizer = WordNetLemmatizer()
    lemitized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

    ## Apply Stemming (Find the roots of similar words)
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in lemitized_words]

    return stemmed_words

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Josh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Josh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Josh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:

vocab = pd.read_csv("reduced_vocabulary.csv")
#vocab = vocab['word'].tolist()
encodings = []
efforts = tickets['effort(s)'].to_numpy()

## Count tickets

descs = tickets['desc'].to_numpy()
for desc in descs:
    parsed = parse_text(desc)
    encoding = bagOfWordsEncoder(vocab, parsed)
    encodings.append(encoding)


In [40]:

## Gather training data
from sklearn.model_selection import train_test_split
labels = pd.read_csv("labelled_dataset.csv")

labels = labels['Grade'].tolist()
labels = [(label - 1)/2 for label in labels]

data = {'encoding': encodings, 'effort': labels}
df = pd.DataFrame(data)

## simplify data ~ 10000
df_low_effort = df[df.effort == 0.0].sample(19000)
df_med_effort = df[df.effort == 0.5].sample(19000)
df_hig_effort = df[df.effort == 1.0].sample(19000)

print(len(df_low_effort))
print(len(df_med_effort))
print(len(df_hig_effort))

result = pd.concat([df_low_effort, df_med_effort, df_hig_effort], axis=0)

train, test = train_test_split(result, test_size=0.2, shuffle=True)

x_train = tf.convert_to_tensor(train['encoding'].to_list())
y_train = tf.convert_to_tensor(train['effort'])

x_test = tf.convert_to_tensor(test['encoding'].to_list())
y_test = tf.convert_to_tensor(test['effort'])

19000
19000
19000


In [44]:
## SETUP NEURAL NETWORK

from tensorflow import keras

model = keras.Sequential([
    keras.layers.Flatten(input_shape=(2742,1)),
    
    keras.layers.Dense(256, activation="relu"),
    #keras.layers.Dropout(0.2),
    keras.layers.Dense(256, activation="relu"),
    #keras.layers.Dropout(0.2),
    keras.layers.Dense(512, activation="relu"),

    keras.layers.Dense(1)
])

model.compile(optimizer=keras.optimizers.Adam(lr=0.1), loss="mean_squared_error", metrics=["accuracy"])

model.fit(x_train, y_train, epochs=20, shuffle=True)

test_loss, test_acc = model.evaluate(x_test, y_test)

print("Tested Acc: ", test_acc)

Epoch 1/20
1425/1425 [==============================] - 13s 8ms/step - loss: 0.1617 - accuracy: 0.4258
Epoch 2/20
1425/1425 [==============================] - 14s 10ms/step - loss: 0.1266 - accuracy: 0.5005
Epoch 3/20
1425/1425 [==============================] - 16s 11ms/step - loss: 0.0989 - accuracy: 0.5606
Epoch 4/20
1425/1425 [==============================] - 15s 11ms/step - loss: 0.0664 - accuracy: 0.6153
Epoch 5/20
1425/1425 [==============================] - 16s 11ms/step - loss: 0.0462 - accuracy: 0.6397
Epoch 6/20
1425/1425 [==============================] - 16s 11ms/step - loss: 0.0351 - accuracy: 0.6503
Epoch 7/20
1425/1425 [==============================] - 15s 10ms/step - loss: 0.0305 - accuracy: 0.6542
Epoch 8/20
1425/1425 [==============================] - 14s 10ms/step - loss: 0.0265 - accuracy: 0.6558
Epoch 9/20
1425/1425 [==============================] - 14s 10ms/step - loss: 0.0235 - accuracy: 0.6562
Epoch 10/20
1425/1425 [==============================] - 14s 10ms